In [4]:
from bs4 import BeautifulSoup
import json
import os
import sqlite3
import pandas as pd
from 

In [46]:
from texts import text_cleaner

ModuleNotFoundError: No module named 'texts'

In [11]:
conn = sqlite3.connect('../../diplom.db')
# cur = conn.cursor()

In [34]:
izvestia = pd.read_sql("SELECT * FROM IZVESTIA_NEW;""", conn)

In [42]:
izvestia["DDATE"] = pd.to_datetime(izvestia["DDATE"]).dt.tz_localize(None) 

In [44]:
izvestia0021 = izvestia[(izvestia.DDATE>="2000-01-01")&(izvestia.DDATE<="2021-09-17")]

In [45]:
izvestia0021["CLEAN_TEXT"] = izvestia0021["TRANSCRIPT"].apply()

,ID,AUTHOR,DDATE,URL,HEADER,SECTION,FILENAME,TRANSCRIPT
2,3,,2018-07-02 13:11:04,https://iz.ru/762225/photo/gudrun5,gudrun5,"Мир, Фото",10000.html,Поделиться:\n
3,4,Алексей Забродин,2018-02-21 18:50:00,https://iz.ru/711745/aleksei-zabrodin/alpiiska...,Альпийская разновекторность,"Мир, Статья",100000.html,Министр иностранных дел России Сергей Лавров п...
5,6,"Ирина Рудевич, Алексей Рудевич",2018-10-29 08:00:00,https://iz.ru/805131/irina-rudevich-aleksei-ru...,Персональные данные. Валюта XXI века,"Финансы, Статья",100003.html,Персональные данные человека — это ценность. С...
6,7,,2020-04-01 13:48:41,https://iz.ru/994423/video/situatciia-s-appara...,Ситуация с аппаратами ИВЛ в стране стабильна,"Общество, Видео",100004.html,Выделить главное\n\n\n\t\t\tвкл\n\t\t\n\n\n\t\...
7,8,Евгений Девятьяров,2017-08-10 21:01:00,https://iz.ru/630974/evgenii-deviatiarov/polet...,Над Крымским мостом запретили летать ниже кило...,"Общество, Статья",100005.html,Министерство транспорта России официально подт...
...,...,...,...,...,...,...,...,...
268541,268542,,2018-05-24 18:56:09,https://iz.ru/747610/photo/kino1,kino1,"Культура, Фото",99992.html,Поделиться:\n
268543,268544,,2021-04-01 19:27:25,https://iz.ru/1145603/photo/tceny-prosiatsia-v...,Цены просятся в полет1,"Экономика, Фото",99994.html,Поделиться:\n
268544,268545,"Михаил Кудин, Алексей Фомин",2020-11-25 19:15:14,https://iz.ru/1092205/mikhail-kudin-aleksei-fo...,Игрок в полях Господних: не стало Диего Марадоны,"Мир, Статья",99996.html,"Сегодня, 25 ноября, футбольный мир потерял сво..."
268546,268547,,2018-11-29 10:01:05,https://iz.ru/818008/photo/11ethnographictouri...,11_ethnographic_tourism_TASS_1242014,"Туризм, Фото",99998.html,Поделиться:\n


In [29]:
pd.to_datetime(izvestia[:5]["DDATE"])

0   2022-07-11 19:12:04+03:00
1   2022-06-03 11:53:45+03:00
2   2018-07-02 16:11:04+03:00
3   2018-02-21 21:50:00+03:00
4   2023-10-23 10:32:15+03:00
Name: DDATE, dtype: datetime64[ns, UTC+03:00]

In [19]:
gd["HEADER"] = pd.to_datetime(gd.HEADER)

In [20]:
gd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   HEADER      1462 non-null   datetime64[ns]
 1   GD_URL      1462 non-null   object        
 2   TRANSCRIPT  1462 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 34.4+ KB


In [3]:
os.chdir("E:\diplom\web pages\izvestia")

In [4]:
def filename():
    for root, dirs, files in os.walk(os.getcwd()):
        for name in files:
            yield name

In [5]:
errors = []
err_files = []

In [15]:
for file in filename():
    with open(file, encoding='utf-8') as f:
        try:
            html = f.read()
            soup = BeautifulSoup(html)
            try:
                author = soup.find(property="article:author").attrs['content'].replace("'", '.')
            except:
                author = ''
            try:
                date =soup.find(property="article:published_time").attrs['content'].replace("'", '.')
            except:
                date = ''
            try:
                url = soup.find(property="og:url").attrs['content'].replace("'", '.')
            except:
                url = ''
            try: 
                header = soup.find(property="og:title").attrs['content'].replace("'", '.')
            except:
                header = ''
            try: 
                section = soup.find(property="article:section").attrs['content'].replace("'", '.')
            except:
                section = ''    
            text = ''
            for i in soup.findAll(itemprop="articleBody"):
                text += i.text.strip().replace("'", '.') + '\n'
            cur.execute(f"""INSERT INTO IZVESTIA_NEW (AUTHOR, DDATE, URL, HEADER, SECTION, FILENAME, TRANSCRIPT) VALUES('{author}', '{date}', '{url}', '{header}', '{section}', '{file}', '{text}');""")
            # conn.commit()
        except Exception as e:
            print(file)
            print(e)
            errors.append(e)
            err_files.append(file)
conn.commit()

In [16]:
1+1

2

In [6]:
res = cur.execute("""SELECT * FROM IZVESTIA""")

In [13]:
cur.execute("""CREATE TABLE IZVESTIA_NEW (
    ID integer primary key autoincrement,
	AUTHOR VARCHAR(200),
    DDATE VARCHAR(200),
    URL VARCHAR(200),
	HEADER VARCHAR(200),
	SECTION VARCHAR(200),
	FILENAME VARCHAR(20),
	TRANSCRIPT TEXT
);""")

In [14]:
conn.commit()

In [18]:
conn.close()

In [2]:
from bs4 import BeautifulSoup
import json
import os
import sqlite3
import nltk
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

ImportError: cannot import name 'Mapping' from 'collections' (C:\Program Files\Python312\Lib\collections\__init__.py)

In [4]:
import os
os.chdir("..")

In [6]:
os.chdir("..")
os.getcwd()

'C:\\Users\\alexander\\PycharmProjects\\diplom'

In [ ]:
!pip instal

In [3]:
!pip install collections --upgrade

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


In [2]:
conn = sqlite3.connect('E:\diplom\diplom.db')
cur = conn.cursor()

In [6]:
# cur.execute("""CREATE TABLE IZVESTIA_COPY (
# ID integer primary key autoincrement,
# 	AUTHOR VARCHAR(200),
#     DDATE VARCHAR(200),
#     URL VARCHAR(200),
# 	HEADER VARCHAR(200),
# 	SECTION VARCHAR(200),
# 	FILENAME VARCHAR(20),
# 	TRANSCRIPT TEXT
# );""")
# cur.execute("""DROP TABLE IZVESTIA_COPY""")
conn.commit()

In [7]:
nlp = spacy.load('ru_core_news_sm')
res = cur.execute("""select * from IZVESTIA_NEW""")
def get_texts():
    for r in res:
        yield r

izvestia_frame = pd.DataFrame(columns=['file', 'date', 'count'])
izvestia_texts_democracy = []
corpus_izvestia = []

for row in get_texts():
    i, author, date, url, header, section, file, text = row
    doc = nlp(text)
    democracy_count = 0
    lemmas = [token.lemma_ for token in doc]
    democracy_count = lemmas.count('демократия')
    izvestia_frame.loc[i] = file, date, democracy_count
#     print(democracy_count)
    if democracy_count:
        izvestia_texts_democracy.append(text)
        corpus_izvestia.append(lemmas)
print('detected democracy texts')
izvestia_frame.to_excel("izvestia.xlsx")
model = Word2Vec(corpus_izvestia, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv

detected democracy texts


window=5

In [8]:
model.get_vector('демократия')

array([-3.5766926 , -3.0316958 ,  0.9686452 , -1.3188351 ,  2.546759  ,
       -0.6474348 ,  0.09102358, -0.45753068, -0.70596963,  1.4003515 ,
        0.46409538,  2.400015  , -0.3453278 , -1.0255783 , -1.0174031 ,
       -0.76842135,  1.3332396 ,  0.08861988,  0.92493194, -2.474383  ,
       -1.0612268 ,  2.1174538 ,  1.8337611 , -0.6436659 , -0.47902954,
        0.71494097,  0.46093917,  1.6548154 ,  1.2463144 , -0.66734   ,
       -2.7372336 ,  2.6495342 ], dtype=float32)

In [9]:
model.most_similar('демократия')

[('ценность', 0.8455156087875366),
 ('справедливость', 0.8448948860168457),
 ('народ', 0.794706404209137),
 ('принцип', 0.783704400062561),
 ('общество', 0.7612237930297852),
 ('либеральный', 0.7468475699424744),
 ('свобода', 0.7453266978263855),
 ('меньшинство', 0.7419408559799194),
 ('движение', 0.7358075380325317),
 ('нация', 0.7312542796134949)]

In [37]:
model.most_similar(positive=['демократия', 'гражданский'], negative=['выбор'])

[('либеральный', 0.7222791910171509),
 ('свободный', 0.6692877411842346),
 ('продвижение', 0.6299036741256714),
 ('социальный', 0.6255321502685547),
 ('современный', 0.6164387464523315),
 ('демократический', 0.6113343238830566),
 ('культурный', 0.6105185747146606),
 ('справедливость', 0.6072991490364075),
 ('процветание', 0.5998404622077942),
 ('двойной', 0.5981652736663818)]

In [26]:
model = Word2Vec(corpus_izvestia, 
                 vector_size=64,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=2).wv

In [27]:
model.most_similar('демократия')

[('ценность', 0.8228520154953003),
 ('принцип', 0.7854652404785156),
 ('справедливость', 0.7762071490287781),
 ('либеральный', 0.7531120181083679),
 ('меньшинство', 0.7300997972488403),
 ('общество', 0.7276099324226379),
 ('единство', 0.7206049561500549),
 ('цивилизация', 0.7091200351715088),
 ('свобода', 0.7019317150115967),
 ('движение', 0.6981418132781982)]

In [42]:
model = Word2Vec(corpus_izvestia, 
                 vector_size=1024,      # embedding vector size
                 min_count=1,  # consider words that occured at least 5 times
                 window=3).wv

In [43]:
model.most_similar('демократия')

[('ценность', 0.8878699541091919),
 ('общество', 0.8470258116722107),
 ('принцип', 0.8460907340049744),
 ('справедливость', 0.845017671585083),
 ('народ', 0.823380708694458),
 ('стандарт', 0.8123600482940674),
 ('движение', 0.8115740418434143),
 ('либеральный', 0.7893827557563782),
 ('меньшинство', 0.783093273639679),
 ('свобода', 0.7830301523208618)]

In [63]:
model.most_similar('путин')

[('зеленский', 0.9247225522994995),
 ('зеленского', 0.8692907094955444),
 ('евсеев', 0.8473546504974365),
 ('васильев', 0.8473291397094727),
 ('макей', 0.8345725536346436),
 ('жириновский', 0.8300738334655762),
 ('зеленским', 0.8071434497833252),
 ('владимир', 0.792557954788208),
 ('лукашенко', 0.7895982265472412),
 ('набоков', 0.7876448035240173)]

In [69]:
model.most_similar(positive=['навальный'])

[('фбр', 0.9039924740791321),
 ('алексей', 0.899131715297699),
 ('ярошенко', 0.8960999250411987),
 ('цру', 0.8897567987442017),
 ('начальник', 0.8874965310096741),
 ('джон', 0.8872145414352417),
 ('виктор', 0.8832811713218689),
 ('моравецкий', 0.8816667795181274),
 ('марко', 0.8805704116821289),
 ('милан', 0.8765102624893188)]

In [74]:
model.most_similar(positive=['навальный'], negative=['деньга'])

[('главред', 0.7952739596366882),
 ('данилюк', 0.7803899049758911),
 ('мильчакова', 0.7721138596534729),
 ('файнер', 0.7712170481681824),
 ('бречалов', 0.7667205333709717),
 ('баканов', 0.753603458404541),
 ('базума', 0.7476952075958252),
 ('фезер', 0.7417121529579163),
 ('gmf', 0.7397336959838867),
 ('мелконян', 0.7363455295562744)]

In [64]:
from gensim.corpora.dictionary import Dictionary

In [65]:
import re

In [66]:
pattern = re.compile('[^a-zA-Z0-9а-яА-ЯёЁ\s]+')

In [41]:
phrases = Phrases(corpus_izvestia, min_count=1, threshold=1)

In [11]:
with open('izvestia_tokens.txt', 'a+', encoding='utf8') as f:
    
    for line in corpus_izvestia:
        f.write(';'.join(line))

In [12]:
izvestia_frame[izvestia_frame['count']>0]

,file,date,count
714,101108.html,2023-04-27T00:00:01+03:00,1
952,101499.html,2017-10-24T09:00:00+03:00,1
1567,10243.html,2023-10-14T00:01:00+03:00,1
1696,10262.html,2023-06-19T19:06:01+03:00,1
3476,105268.html,2019-01-29T19:15:00+03:00,1
...,...,...,...
265066,94923.html,2017-09-25T08:00:00+03:00,4
267223,98091.html,2023-03-20T00:01:00+03:00,2
267227,98095.html,2021-01-09T00:02:00+03:00,1
267983,99177.html,2017-08-09T14:01:00+03:00,1


In [13]:
izvestia_frame['datetime'] = izvestia_frame['date']

In [25]:
pd.to_datetime(izvestia_frame['datetime'], errors='coerce', utc=False).astype('datetime64')

ValueError: Tz-aware datetime.datetime cannot be converted to datetime64 unless utc=True

In [22]:
pd.to_datetime(izvestia_frame['datetime'], format='%Y-%m-%dT%H:%M:%S+03:00').dt

ValueError: time data '' does not match format '%Y-%m-%dT%H:%M:%S+03:00' (match)

In [18]:
izvestia_frame['date'] = pd.to_datetime(izvestia_frame['datetime']).dt.date

AttributeError: Can only use .dt accessor with datetimelike values

In [40]:
from gensim.models.phrases import Phrases

In [1]:
from bs4 import BeautifulSoup
import json
import os
import sqlite3
import nltk
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from pymystem3 import Mystem
import pickle
import re

In [2]:
conn = sqlite3.connect('D:\diplom main\diplom.db')
cur = conn.cursor()
df = pd.read_sql("""SELECT * FROM IZVESTIA_TEXTS0021""", conn)
conn.close()
pattern = re.compile('[^a-zA-Z0-9а-яА-ЯёЁ\s]+')
stopwords_rus = nltk.corpus.stopwords.words('russian')

def text_cleaner(text):
    clean_text = ' '.join(list(filter(None, [i.strip() for i in re.sub(pattern, ' ', text).lower().split(' ') if i not in stopwords_rus])))
    out = ' '.join([i for i in Mystem().lemmatize(clean_text) if i not in (' ', '\n')])
    return out
def count_democracy(text):
    return text.count('демократия')

In [3]:
df.shape

(155560, 9)

In [4]:
df.head()

,index,ID,AUTHOR,DDATE,URL,HEADER,SECTION,FILENAME,TRANSCRIPT
0,0,3,,2018-07-02,https://iz.ru/762225/photo/gudrun5,gudrun5,"Мир, Фото",10000.html,Поделиться:\n
1,1,4,Алексей Забродин,2018-02-21,https://iz.ru/711745/aleksei-zabrodin/alpiiska...,Альпийская разновекторность,"Мир, Статья",100000.html,Министр иностранных дел России Сергей Лавров п...
2,2,6,"Ирина Рудевич, Алексей Рудевич",2018-10-29,https://iz.ru/805131/irina-rudevich-aleksei-ru...,Персональные данные. Валюта XXI века,"Финансы, Статья",100003.html,Персональные данные человека — это ценность. С...
3,3,7,,2020-04-01,https://iz.ru/994423/video/situatciia-s-appara...,Ситуация с аппаратами ИВЛ в стране стабильна,"Общество, Видео",100004.html,Выделить главное\n\n\n\t\t\tвкл\n\t\t\n\n\n\t\...
4,4,8,Евгений Девятьяров,2017-08-10,https://iz.ru/630974/evgenii-deviatiarov/polet...,Над Крымским мостом запретили летать ниже кило...,"Общество, Статья",100005.html,Министерство транспорта России официально подт...


In [5]:
dfArticles = df[df.SECTION.str.contains('Статья')]

In [6]:
dfArticles.shape

(35551, 9)

In [ ]:
dfArticles['TOKENIZED_TRANSCRIPT'] = dfArticles['TRANSCRIPT'].apply(text_cleaner)

In [ ]:
dfArticles["COUNT_DEMOCRACY"] = dfArticles['TOKENIZED_TRANSCRIPT'].apply(count_democracy)

In [ ]:
dfArticles.to_csv("izvestia.csv")

In [ ]:
conn = sqlite3.connect('D:\diplom main\diplom.db')
dfArticles.to_sql("IZVESTIA_TOKENIZED", conn)
conn.commit()

In [ ]:
1